In [1]:
%matplotlib inline
%env KERAS_BACKEND=tensorflow

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import common

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.losses import mean_squared_error
from keras.losses import mean_absolute_percentage_error
from keras.callbacks import TensorBoard

from sklearn import preprocessing
from sklearn import metrics

env: KERAS_BACKEND=tensorflow


Using TensorFlow backend.


In [3]:
import importlib
importlib.reload(common)

<module 'common' from 'c:\\Users\\Surya\\Desktop\\work\\notebooks\\common.py'>

In [4]:
df = common.load_df("../data/insead", "*.csv")
print(common.load_df.cache_info())

# process. at last, normalize.
_cols = ["drybulb", "loadsys", "rh", "systotpower"]
df = common.Process.replace_nulls(df, cols=_cols)
df = common.Process.replace_with_near(df, cols=_cols)
df = common.Process.smooth_data(df, cols=_cols)
df = common.Process.get_normalized_df(df, scale=(0.1, 1), cols=_cols)

# add few more features.
df["weekday"] = df.index.weekday
df["weekend"] = df.weekday.apply(lambda x: int(x in (5,6)))
df["hour"] = df.index.hour

CacheInfo(hits=0, misses=1, maxsize=None, currsize=1)


## Experiment 1
```
X = DB, Rh, Hour, Weekday
y = Loadsys

Train: Mid Jun-Jul
Test: Mid Jul-Aug
```

In [7]:
# manually select training and validaiton data.
batch_size = 250
train_df = df["2016-06-15":"2016-07-14"]
validation_df = df["2016-07-15":"2016-08-14"]

# The recipe.
features = ["drybulb", "rh", "hour", "weekend"]
target = ["loadsys"]
lookback = 15

# Prepare features and target vectors
train_x, train_y = common.prepare_keras_data(train_df, features, target, lookback, batch_size)
validation_x, validation_y = common.prepare_keras_data(validation_df, features, target, lookback, batch_size)
print(train_x.shape, train_y.shape)

(43000, 1, 60) (43000, 1, 1)


In [8]:
model = Sequential([
    LSTM(60, batch_input_shape=(batch_size, train_x.shape[1], train_x.shape[2]), stateful=True, return_sequences=True),
    LSTM(120, stateful=True, return_sequences=True),
    LSTM(120, stateful=True, return_sequences=True),
    LSTM(80, stateful=True, return_sequences=True),
    LSTM(40, stateful=True, return_sequences=True),
    LSTM(10, stateful=True, return_sequences=True),
    Dense(1)
])

model.compile(
    loss=mean_squared_error,
    optimizer="adam",
    metrics=["accuracy", mean_absolute_percentage_error]
)

for epoch in range(151):
    history = model.fit(
        x=train_x,
        y=train_y,
        batch_size=batch_size,
        verbose=0,
        shuffle=False,
        epochs=1,
        validation_data=(validation_x, validation_y),
        callbacks=[TensorBoard(log_dir="../output/keras15")])
    model.reset_states()
    if epoch % 50 == 0:
        print("At epoch ", epoch)

At epoch  0
At epoch  50
At epoch  100
At epoch  150


## Experiment 2

```
y = systotpower
x = loadsys, weekend, hour, db, rh
```

In [9]:
# The recipe.
features = ["drybulb", "rh", "hour", "weekend", "loadsys"]
target = ["systotpower"]
lookback = 15

# Prepare features and target vectors
train_x, train_y = common.prepare_keras_data(train_df, features, target, lookback, batch_size)
validation_x, validation_y = common.prepare_keras_data(validation_df, features, target, lookback, batch_size)
print(train_x.shape, train_y.shape)

(43000, 1, 75) (43000, 1, 1)


In [10]:
model = Sequential([
    LSTM(75, batch_input_shape=(batch_size, train_x.shape[1], train_x.shape[2]), stateful=True, return_sequences=True),
    LSTM(120, stateful=True, return_sequences=True),
    LSTM(120, stateful=True, return_sequences=True),
    LSTM(80, stateful=True, return_sequences=True),
    LSTM(40, stateful=True, return_sequences=True),
    LSTM(10, stateful=True, return_sequences=True),
    Dense(1)
])

model.compile(
    loss=mean_squared_error,
    optimizer="adam",
    metrics=["accuracy", mean_absolute_percentage_error]
)

for epoch in range(151):
    history = model.fit(
        x=train_x,
        y=train_y,
        batch_size=batch_size,
        verbose=0,
        shuffle=False,
        epochs=1,
        validation_data=(validation_x, validation_y),
        callbacks=[TensorBoard(log_dir="../output/keras16")])
    model.reset_states()
    if epoch % 50 == 0:
        print("At epoch ", epoch)

At epoch  0
At epoch  50
At epoch  100
At epoch  150
